In [1]:
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense,Flatten,Dropout,BatchNormalization,Conv2D,MaxPool2D
from keras.callbacks import ModelCheckpoint,EarlyStopping,ReduceLROnPlateau
from keras.optimizers import Adam
import os
import cv2
import numpy as np
from time import sleep
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model

model = load_model("F:/Computers/Python/EmotionsRecognitionModel/emotions.h5")

class_labels = {0: 'Angry',1: 'Fear',2: 'Happy',3: 'Neutral',4: 'Sad',5: 'Surprise'}
print(class_labels)

Using TensorFlow backend.


{0: 'Angry', 1: 'Fear', 2: 'Happy', 3: 'Neutral', 4: 'Sad', 5: 'Surprise'}


In [2]:
face_classifier = cv2.CascadeClassifier("C:/Users/Abhinav/Face-Recognition-master/Haarcascades/haarcascade_frontalface_default.xml") # CHANGE THE LOCATION TO THE PATH OF HAR CASCADE FILE

def face_detector(img):
    
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray,1.3,5)
    if faces is ():
        return (0,0,0,0), np.zeros((48,48),np.uint8), img

    for (x,y,w,h) in faces:
        y = y-20
        h = h+50
        cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2)
        roi_gray = gray[y:y+h,x:x+w]

    try:
        roi_gray = cv2.resize(roi_gray,(48,48),interpolation=cv2.INTER_AREA)

    except:
        return (x,w,y,h), np.zeros((48,48),np.uint8),img
    return (x,y,w,h),roi_gray,img


cap = cv2.VideoCapture(0)

while True:
    
    ret,frame = cap.read()
    frame = cv2.flip(frame,1)
    (x,y,w,h),face,image = face_detector(frame)
    
    if np.sum([face]) != 0.0:
        
        roi = face.astype('float') / 255.0
        roi = img_to_array(roi)
        roi = np.expand_dims(roi, axis=0)

        preds = model.predict(roi)[0]
        label = class_labels[preds.argmax()]
        
        cv2.putText(image, label, (x,y), cv2.FONT_HERSHEY_SIMPLEX,1,(0,255,0),2)
        
    else:
        
        cv2.putText(image,"No face found", (20,60), cv2.FONT_HERSHEY_SIMPLEX,1,(0,255,0),2)

    cv2.imshow("Emotions",image)
    
    if cv2.waitKey(1)==13:
        break
        
cap.release()
cv2.destroyAllWindows()